### Imports

In [1]:
import numpy as np
import os,cv2,math,glob,random
import scipy.io as sio
from math import cos, sin
from pathlib import Path
import pandas as pd
import mediapipe
import warnings
import matplotlib.pyplot as plt
warnings.filterwarnings('ignore')
import time

objc[14095]: Class CaptureDelegate is implemented in both /Users/omarmarie/miniforge3/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x14650e538) and /Users/omarmarie/miniforge3/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x14b1c8860). One of the two will be used. Which one is undefined.
objc[14095]: Class CVWindow is implemented in both /Users/omarmarie/miniforge3/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x14650e588) and /Users/omarmarie/miniforge3/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x149fd8a68). One of the two will be used. Which one is undefined.
objc[14095]: Class CVView is implemented in both /Users/omarmarie/miniforge3/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x14650e5b0) and /Users/omarmarie/miniforge3/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x149fd8a90). One of the two will be used. Which one is undefined.
objc[14095]: Class CVSlider is implemented in both /U

### Read all images

In [2]:
ls

0. Getting Started.ipynb
0.1 Explore a Demo.ipynb
1. Read and Prepare Data.ipynb
2. Preprocess data - Center and Normalize.ipynb
3. Train Models - Random Forrest.ipynb
AFLW2000/
AFLW2000-3D.zip
face_contours_and_irises.png
face_tesselation_only.png
implement.ipynb
labels.csv
out.avi
out.mp4
outimg.jpg
outpy2.avi
pitch_model.pkl
prepared_data.csv
preprocessed_data.csv
roll_model.pkl
screen.mp4
test.jpg
yaw_model.pkl


In [3]:
path = 'AFLW2000'

In [4]:
import glob
arr_images = []
arr_mat    = []

for filename in glob.iglob(path + '/*.jpg', recursive=True):
    arr_images.append(filename)
    
for matname in glob.iglob(path + '/*.mat', recursive=True):
    arr_mat.append(matname)

In [5]:
len(arr_images), len(arr_mat)

(2000, 2000)

In [6]:
arr_images.sort() 
arr_mat.sort()

We have 200 images in the directory with their mat files

### Extract yaw, roll and pitch from each image along with data points

In [7]:
data_points = []
yaw_arr     = []
roll_arr    = []
pitch_arr   = []

In [8]:
faceModule = mediapipe.solutions.face_mesh
faces = faceModule.FaceMesh(static_image_mode=True)

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


In [9]:
# Extract yaw, roll, pitch
for filename in arr_images:
   
    # Read mat file name
    mat_name = filename.replace(".jpg", ".mat")

    # Load matfile names
    mat_file = sio.loadmat(mat_name)
    image    = cv2.imread(filename)

    # extracting the labels 3 angels
    pose_para = mat_file["Pose_Para"][0][:3]
    results = faces.process(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    if results.multi_face_landmarks is not None:
        pitch = pose_para[0] * 180 / np.pi
        yaw = pose_para[1]   * 180 / np.pi
        roll = pose_para[2]  * 180 / np.pi

        # Append to the list
        pitch_arr.append(pitch)
        yaw_arr.append(yaw)
        roll_arr.append(roll)
        
        for face in results.multi_face_landmarks:
            for landmark in face.landmark:
                data_points.append(landmark.x * 450)
                data_points.append(landmark.y * 450)

### Save in Pandas dataframe

In [10]:
# Save all the targets in pandas dataframe
targets = np.array([yaw_arr, pitch_arr, roll_arr], dtype=np.float32)
df      = pd.DataFrame(targets.T,columns=['yaw','pitch','roll'])
df.head()

,yaw,pitch,roll
0,1.044306,-22.874239,4.908885
1,68.155243,26.932743,17.243670
2,50.485413,-10.579652,-13.570645
3,17.143373,-10.048455,-21.392782
4,0.685565,-1.536199,-12.643008


In [11]:
# Save all data points in pandas dataframe
points = np.array(data_points, dtype=np.float32)
points = points.reshape(1850, 936)
df_2   = pd.DataFrame(points)

In [12]:
df_2.head()

,0,1,2,3,4,5,6,7,8,9,...,926,927,928,929,930,931,932,933,934,935
0,218.500336,309.087372,220.162933,287.946686,220.391815,291.276459,215.884537,253.167267,220.785034,279.621368,...,251.919342,218.242798,246.828323,221.712997,243.671173,225.133820,299.597992,212.068924,304.294952,208.753708
1,198.656555,288.920837,187.822037,266.461975,201.997467,274.696259,196.234436,245.241287,187.140442,259.584656,...,235.139984,228.096268,230.158264,230.242477,225.326859,231.676422,263.041138,225.040771,267.364594,222.863190
2,143.050110,359.648987,144.704651,349.490448,143.292709,351.969086,140.051865,335.753998,144.767014,345.590118,...,150.435013,323.004913,148.621979,324.391632,147.877350,325.510345,167.899490,320.828583,169.798294,319.338348
3,226.721100,312.409027,211.268036,294.259033,220.251495,296.429535,199.995224,265.632965,207.779312,286.926544,...,228.403564,225.441040,223.977921,229.782516,220.764420,233.405243,268.021088,205.045059,271.196625,200.103424
4,229.615387,301.073395,221.734436,272.162018,224.898712,281.019989,210.327179,244.071304,219.695938,263.117737,...,234.216660,216.879883,229.828812,219.909836,227.546188,221.720444,271.093933,203.719376,275.063751,201.860153


In [13]:
findal_df = pd.concat([df_2, df], axis=1)

In [14]:
findal_df

,0,1,2,3,4,5,6,7,8,9,...,929,930,931,932,933,934,935,yaw,pitch,roll
0,218.500336,309.087372,220.162933,287.946686,220.391815,291.276459,215.884537,253.167267,220.785034,279.621368,...,221.712997,243.671173,225.133820,299.597992,212.068924,304.294952,208.753708,1.044306,-22.874239,4.908885
1,198.656555,288.920837,187.822037,266.461975,201.997467,274.696259,196.234436,245.241287,187.140442,259.584656,...,230.242477,225.326859,231.676422,263.041138,225.040771,267.364594,222.863190,68.155243,26.932743,17.243670
2,143.050110,359.648987,144.704651,349.490448,143.292709,351.969086,140.051865,335.753998,144.767014,345.590118,...,324.391632,147.877350,325.510345,167.899490,320.828583,169.798294,319.338348,50.485413,-10.579652,-13.570645
3,226.721100,312.409027,211.268036,294.259033,220.251495,296.429535,199.995224,265.632965,207.779312,286.926544,...,229.782516,220.764420,233.405243,268.021088,205.045059,271.196625,200.103424,17.143373,-10.048455,-21.392782
4,229.615387,301.073395,221.734436,272.162018,224.898712,281.019989,210.327179,244.071304,219.695938,263.117737,...,219.909836,227.546188,221.720444,271.093933,203.719376,275.063751,201.860153,0.685565,-1.536199,-12.643008
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1845,231.767944,307.775665,239.631592,281.503967,233.323410,287.865509,230.158554,251.943588,240.771652,273.285065,...,226.398132,243.003448,228.730682,283.061462,215.789749,286.082703,213.496033,-16.261791,-17.552992,2.208994
1846,244.801254,310.453735,254.393921,285.928253,247.208054,291.115692,245.549637,253.257172,255.840897,277.273621,...,225.096268,258.799957,227.892044,296.994995,215.894379,300.578857,211.868317,-24.621336,-21.058870,7.035404
1847,259.925873,303.034760,267.883087,281.693115,263.931854,289.061279,269.205811,255.693802,270.546997,274.189850,...,245.582626,292.356171,246.245621,331.897308,252.047241,336.482086,248.884262,32.493248,-8.940119,-6.218641
1848,221.296005,311.795471,225.353119,286.964539,223.831024,292.440887,220.586487,251.742584,226.611496,277.857452,...,223.459534,247.642944,226.166061,303.265503,216.790451,308.045959,213.362595,-4.035367,-11.293093,6.022806


In [15]:
findal_df.to_csv('prepared_data.csv', index=False)